# 📊 Model Validation & Performance Tracker

**Purpose**: Automatically evaluate scanner performance across multiple holding periods (5/7/10/14 days) and track model quality over time.

## Key Features
- **Multi-Period Backtesting**: Compare hit rates at T+5, T+7, T+10, T+14
- **Strategy Comparison**: Weekly Top 5 vs Pro30 vs Movers
- **Trend Analysis**: Is the model improving or degrading?
- **Factor Attribution**: Which scoring factors predict success?
- **Automated Daily Testing**: Run after each scan

In [1]:
from __future__ import annotations

import json
import logging
import re
import warnings
from dataclasses import dataclass
from datetime import datetime, timedelta
from pathlib import Path
from typing import Any, Optional

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf

# Load environment variables from .env
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")
logger = logging.getLogger(__name__)

# Configuration
OUTPUTS_ROOT = Path("outputs")
HOLDING_PERIODS = [5, 7, 10, 14]  # Days to test
HIT_THRESHOLDS = [5.0, 10.0, 15.0]  # % gain thresholds
PRIMARY_THRESHOLD = 10.0  # Main KPI: +10%
PRIMARY_PERIOD = 7  # Main KPI: 7 trading days

print(f"✅ Configuration loaded")
print(f"   Holding periods: {HOLDING_PERIODS} days")
print(f"   Hit thresholds: {HIT_THRESHOLDS}%")
print(f"   Primary KPI: +{PRIMARY_THRESHOLD}% within {PRIMARY_PERIOD} days")

✅ Configuration loaded
   Holding periods: [5, 7, 10, 14] days
   Hit thresholds: [5.0, 10.0, 15.0]%
   Primary KPI: +10.0% within 7 days


In [2]:
# ═══════════════════════════════════════════════════════════════════════════════
# DATA LOADING UTILITIES
# ═══════════════════════════════════════════════════════════════════════════════

_DATE_RE = re.compile(r"^\d{4}-\d{2}-\d{2}$")


def _dedup_keep_order(items) -> list[str]:
    """Deduplicate while preserving order."""
    out, seen = [], set()
    for x in items:
        t = str(x).strip().upper()
        if t and t not in seen:
            out.append(t)
            seen.add(t)
    return out


def _safe_json(path: Path) -> dict:
    """Safely load JSON file."""
    if not path.exists():
        return {}
    try:
        return json.loads(path.read_text(encoding="utf-8"))
    except Exception:
        return {}


def _safe_csv(path: Path) -> pd.DataFrame:
    """Safely load CSV file."""
    if not path.exists():
        return pd.DataFrame()
    try:
        return pd.read_csv(path)
    except Exception:
        return pd.DataFrame()


@dataclass
class DatePicks:
    """Picks from a single scan date."""
    date_str: str
    weekly_top5: list[str]
    pro30: list[str]
    movers: list[str]
    combined: list[str]
    metadata: dict  # scores, ranks, etc.


def iter_output_dates(root: Path = OUTPUTS_ROOT) -> list[str]:
    """Get all available scan dates."""
    if not root.exists():
        return []
    return sorted([p.name for p in root.iterdir() if p.is_dir() and _DATE_RE.match(p.name)])


def load_picks_for_date(date_str: str, root: Path = OUTPUTS_ROOT) -> DatePicks:
    """Load all picks and metadata for a single date."""
    run_dir = root / date_str
    metadata = {}
    
    # Weekly Top 5
    weekly = []
    top5_json = run_dir / f"weekly_scanner_top5_{date_str}.json"
    if top5_json.exists():
        obj = _safe_json(top5_json)
        for x in obj.get("top5", []):
            if isinstance(x, dict) and x.get("ticker"):
                ticker = str(x["ticker"]).strip().upper()
                weekly.append(ticker)
                # Store metadata for factor analysis
                metadata[ticker] = {
                    "source": "weekly_top5",
                    "rank": len(weekly),
                    "composite_score": x.get("composite_score"),
                    "technical_score": x.get("technical_score"),
                    "catalyst_score": x.get("catalyst_score"),
                    "name": x.get("name", ""),
                }
    else:
        # Fallback: hybrid_analysis
        hybrid = _safe_json(run_dir / f"hybrid_analysis_{date_str}.json")
        for x in hybrid.get("weekly_top5", []):
            if isinstance(x, dict) and x.get("ticker"):
                ticker = str(x["ticker"]).strip().upper()
                weekly.append(ticker)
                metadata[ticker] = {"source": "weekly_top5", "rank": len(weekly)}
    
    weekly = _dedup_keep_order(weekly)
    
    # Pro30 (momentum, breakout, reversal)
    pro30 = []
    for pattern in ["30d_momentum_candidates", "30d_breakout_candidates", "30d_reversal_candidates"]:
        csv_path = run_dir / f"{pattern}_{date_str}.csv"
        df = _safe_csv(csv_path)
        if not df.empty and "Ticker" in df.columns:
            for _, row in df.iterrows():
                ticker = str(row["Ticker"]).strip().upper()
                if ticker and ticker not in metadata:
                    pro30.append(ticker)
                    metadata[ticker] = {
                        "source": pattern.replace("_candidates", ""),
                        "score": row.get("Score") or row.get("Composite_Score"),
                    }
    pro30 = _dedup_keep_order(pro30)
    
    # Movers
    movers = []
    hybrid = _safe_json(run_dir / f"hybrid_analysis_{date_str}.json")
    for t in hybrid.get("movers_tickers", []):
        ticker = str(t).strip().upper()
        if ticker:
            movers.append(ticker)
            if ticker not in metadata:
                metadata[ticker] = {"source": "movers"}
    movers = _dedup_keep_order(movers)
    
    combined = _dedup_keep_order(weekly + pro30 + movers)
    
    return DatePicks(
        date_str=date_str,
        weekly_top5=weekly,
        pro30=pro30,
        movers=movers,
        combined=combined,
        metadata=metadata,
    )


print(f"✅ Data loading utilities ready")

✅ Data loading utilities ready


In [3]:
# ═══════════════════════════════════════════════════════════════════════════════
# MULTI-PERIOD BACKTESTING ENGINE
# ═══════════════════════════════════════════════════════════════════════════════

import os
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

# Check for Polygon API key
POLYGON_API_KEY = os.getenv("POLYGON_API_KEY")
USE_POLYGON = bool(POLYGON_API_KEY)

if USE_POLYGON:
    print(f"✅ Polygon API key found - using Polygon as primary data source")
else:
    print(f"⚠️ No POLYGON_API_KEY found - using Yahoo Finance only")


@dataclass
class PriceData:
    """Price data for backtesting."""
    close: pd.DataFrame
    high: pd.DataFrame
    low: pd.DataFrame


def _fetch_polygon_daily(ticker: str, start_date: str, end_date: str, api_key: str) -> pd.DataFrame:
    """Fetch daily OHLCV from Polygon for a single ticker."""
    try:
        url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/day/{start_date}/{end_date}"
        resp = requests.get(
            url,
            params={"adjusted": "true", "sort": "asc", "limit": 5000, "apiKey": api_key},
            timeout=10,
        )
        resp.raise_for_status()
        results = resp.json().get("results") or []
        if not results:
            return pd.DataFrame()
        df = pd.DataFrame(results)
        if df.empty or not {"o", "h", "l", "c", "v", "t"}.issubset(df.columns):
            return pd.DataFrame()
        df["Date"] = pd.to_datetime(df["t"], unit="ms")
        df = df.rename(columns={"o": "Open", "h": "High", "l": "Low", "c": "Close", "v": "Volume"})
        df = df.set_index("Date")[["Open", "High", "Low", "Close", "Volume"]].dropna()
        return df
    except Exception:
        return pd.DataFrame()


def _download_polygon_batch(tickers: list[str], start_date: str, end_date: str, api_key: str, max_workers: int = 8) -> dict[str, pd.DataFrame]:
    """Download daily OHLCV for many tickers from Polygon."""
    results: dict[str, pd.DataFrame] = {}
    
    def worker(t: str):
        return t, _fetch_polygon_daily(t, start_date, end_date, api_key)
    
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = {ex.submit(worker, t): t for t in tickers}
        for fut in as_completed(futures):
            t, df = fut.result()
            results[t] = df
    return results


def _download_yfinance(tickers: list[str], start_date: str, end_date: str) -> PriceData:
    """Download from Yahoo Finance (fallback)."""
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)
    
    data = yf.download(
        tickers=tickers,
        start=start_dt,
        end=end_dt,
        progress=False,
        auto_adjust=False,
        threads=True,
        group_by="column",
    )
    
    if data is None or data.empty:
        return PriceData(pd.DataFrame(), pd.DataFrame(), pd.DataFrame())
    
    if isinstance(data.columns, pd.MultiIndex):
        close = data.get("Close", pd.DataFrame()).copy()
        high = data.get("High", pd.DataFrame()).copy()
        low = data.get("Low", pd.DataFrame()).copy()
    else:
        close = data[["Close"]].copy()
        close.columns = [tickers[0]]
        high = data[["High"]].copy()
        high.columns = [tickers[0]]
        low = data[["Low"]].copy()
        low.columns = [tickers[0]]
    
    return PriceData(close.sort_index(), high.sort_index(), low.sort_index())


def download_prices(
    tickers: list[str],
    start_date: str,
    end_date: str,
) -> PriceData:
    """
    Download OHLC data for multiple tickers.
    Uses Polygon as primary source if API key available, falls back to Yahoo Finance.
    """
    tickers = _dedup_keep_order(tickers)
    if not tickers:
        return PriceData(pd.DataFrame(), pd.DataFrame(), pd.DataFrame())
    
    logger.info(f"Downloading prices for {len(tickers)} tickers: {start_date} → {end_date}")
    
    # Try Polygon first if available
    if USE_POLYGON:
        logger.info("Using Polygon.io as primary data source...")
        polygon_data = _download_polygon_batch(tickers, start_date, end_date, POLYGON_API_KEY)
        
        # Build DataFrames from Polygon results
        close_dfs = []
        high_dfs = []
        low_dfs = []
        success_count = 0
        failed_tickers = []
        
        for ticker, df in polygon_data.items():
            if not df.empty:
                close_dfs.append(df[["Close"]].rename(columns={"Close": ticker}))
                high_dfs.append(df[["High"]].rename(columns={"High": ticker}))
                low_dfs.append(df[["Low"]].rename(columns={"Low": ticker}))
                success_count += 1
            else:
                failed_tickers.append(ticker)
        
        logger.info(f"Polygon: {success_count}/{len(tickers)} tickers succeeded")
        
        # Fall back to Yahoo Finance for failed tickers
        if failed_tickers:
            logger.info(f"Falling back to Yahoo Finance for {len(failed_tickers)} tickers...")
            yf_data = _download_yfinance(failed_tickers, start_date, end_date)
            if not yf_data.close.empty:
                for ticker in failed_tickers:
                    if ticker in yf_data.close.columns:
                        close_dfs.append(yf_data.close[[ticker]])
                        high_dfs.append(yf_data.high[[ticker]])
                        low_dfs.append(yf_data.low[[ticker]])
        
        # Combine all data
        if close_dfs:
            close = pd.concat(close_dfs, axis=1).sort_index()
            high = pd.concat(high_dfs, axis=1).sort_index()
            low = pd.concat(low_dfs, axis=1).sort_index()
            return PriceData(close, high, low)
        
        # If Polygon completely failed, fall back to Yahoo Finance
        logger.warning("Polygon failed completely, falling back to Yahoo Finance...")
    
    # Yahoo Finance fallback
    return _download_yfinance(tickers, start_date, end_date)


def compute_forward_returns(
    prices: PriceData,
    ticker: str,
    entry_date: str,
    holding_periods: list[int],
    use_high: bool = True,
) -> dict[int, dict]:
    """
    Compute forward returns for multiple holding periods.
    
    Returns dict: {period: {entry_price, exit_price, max_price, min_price, return_pct, max_return_pct, max_drawdown_pct}}
    """
    results = {}
    ticker = ticker.upper()
    
    if prices.close.empty or ticker not in prices.close.columns:
        return {p: {} for p in holding_periods}
    
    close_s = prices.close[ticker]
    high_s = prices.high[ticker] if ticker in prices.high.columns else close_s
    low_s = prices.low[ticker] if ticker in prices.low.columns else close_s
    
    # Find entry: first valid close on/after entry_date
    entry_ts = pd.Timestamp(entry_date)
    valid_mask = (close_s.index >= entry_ts) & close_s.notna()
    
    if not valid_mask.any():
        return {p: {} for p in holding_periods}
    
    entry_idx = valid_mask.idxmax()
    entry_pos = close_s.index.get_loc(entry_idx)
    entry_price = float(close_s.iloc[entry_pos])
    
    for period in holding_periods:
        # Forward window: entry+1 to entry+period (exclude entry day since we buy at close)
        start_pos = entry_pos + 1
        end_pos = start_pos + period
        
        if start_pos >= len(close_s):
            results[period] = {"entry_price": entry_price, "insufficient_data": True}
            continue
        
        close_window = close_s.iloc[start_pos:end_pos].dropna()
        high_window = high_s.iloc[start_pos:end_pos].dropna() if use_high else close_window
        low_window = low_s.iloc[start_pos:end_pos].dropna()
        
        if close_window.empty:
            results[period] = {"entry_price": entry_price, "insufficient_data": True}
            continue
        
        exit_price = float(close_window.iloc[-1]) if len(close_window) > 0 else None
        max_price = float(high_window.max()) if len(high_window) > 0 else None
        min_price = float(low_window.min()) if len(low_window) > 0 else None
        
        return_pct = ((exit_price / entry_price) - 1) * 100 if exit_price else None
        max_return_pct = ((max_price / entry_price) - 1) * 100 if max_price else None
        max_drawdown_pct = ((min_price / entry_price) - 1) * 100 if min_price else None
        
        results[period] = {
            "entry_price": round(entry_price, 2),
            "exit_price": round(exit_price, 2) if exit_price else None,
            "max_price": round(max_price, 2) if max_price else None,
            "min_price": round(min_price, 2) if min_price else None,
            "return_pct": round(return_pct, 2) if return_pct else None,
            "max_return_pct": round(max_return_pct, 2) if max_return_pct else None,
            "max_drawdown_pct": round(max_drawdown_pct, 2) if max_drawdown_pct else None,
            "trading_days": len(close_window),
        }
    
    return results


print(f"✅ Multi-period backtesting engine ready")

✅ Polygon API key found - using Polygon as primary data source
✅ Multi-period backtesting engine ready


In [4]:
# ═══════════════════════════════════════════════════════════════════════════════
# FULL BACKTEST RUNNER
# ═══════════════════════════════════════════════════════════════════════════════

def run_full_backtest(
    start_date: Optional[str] = None,
    end_date: Optional[str] = None,
    holding_periods: list[int] = HOLDING_PERIODS,
    hit_thresholds: list[float] = HIT_THRESHOLDS,
    min_matured_days: int = 0,  # Only include picks with enough forward data
) -> pd.DataFrame:
    """
    Run comprehensive backtest across all dates and holding periods.
    
    Returns DataFrame with one row per (date, ticker, period) combination.
    """
    all_dates = iter_output_dates()
    
    if start_date:
        all_dates = [d for d in all_dates if d >= start_date]
    if end_date:
        all_dates = [d for d in all_dates if d <= end_date]
    
    if not all_dates:
        logger.warning("No scan dates found in range")
        return pd.DataFrame()
    
    logger.info(f"Found {len(all_dates)} scan dates: {all_dates[0]} → {all_dates[-1]}")
    
    # Load all picks
    all_picks = [load_picks_for_date(d) for d in all_dates]
    
    # Collect all tickers
    all_tickers = _dedup_keep_order([t for p in all_picks for t in p.combined])
    logger.info(f"Total unique tickers: {len(all_tickers)}")
    
    # Download prices (with padding for forward windows)
    max_period = max(holding_periods)
    price_end = (datetime.strptime(all_dates[-1], "%Y-%m-%d") + timedelta(days=max_period * 2)).strftime("%Y-%m-%d")
    today = datetime.now().strftime("%Y-%m-%d")
    price_end = min(price_end, today)
    
    prices = download_prices(all_tickers, all_dates[0], price_end)
    
    if prices.close.empty:
        logger.error("Failed to download price data")
        return pd.DataFrame()
    
    # Compute returns for each (date, ticker, period)
    rows = []
    
    for picks in all_picks:
        date_str = picks.date_str
        weekly_set = set(picks.weekly_top5)
        pro30_set = set(picks.pro30)
        movers_set = set(picks.movers)
        
        for ticker in picks.combined:
            forward_returns = compute_forward_returns(
                prices, ticker, date_str, holding_periods
            )
            
            meta = picks.metadata.get(ticker, {})
            
            for period, ret_data in forward_returns.items():
                if not ret_data or ret_data.get("insufficient_data"):
                    continue
                
                row = {
                    "scan_date": date_str,
                    "ticker": ticker,
                    "period": period,
                    "in_weekly_top5": ticker in weekly_set,
                    "in_pro30": ticker in pro30_set,
                    "in_movers": ticker in movers_set,
                    "weekly_rank": meta.get("rank") if ticker in weekly_set else None,
                    "source": meta.get("source", "unknown"),
                    "composite_score": meta.get("composite_score"),
                    "technical_score": meta.get("technical_score"),
                    "catalyst_score": meta.get("catalyst_score"),
                    **ret_data,
                }
                
                # Add hit flags for each threshold
                for thresh in hit_thresholds:
                    max_ret = ret_data.get("max_return_pct")
                    row[f"hit_{int(thresh)}pct"] = bool(max_ret is not None and max_ret >= thresh)
                
                rows.append(row)
    
    df = pd.DataFrame(rows)
    logger.info(f"Backtest complete: {len(df)} observations")
    
    return df


print(f"✅ Full backtest runner ready")

✅ Full backtest runner ready


In [5]:
# ═══════════════════════════════════════════════════════════════════════════════
# ANALYTICS & AGGREGATIONS
# ═══════════════════════════════════════════════════════════════════════════════

def compute_hit_rate_matrix(df: pd.DataFrame, thresholds: list[float] = HIT_THRESHOLDS) -> pd.DataFrame:
    """
    Compute hit rate matrix: rows = holding periods, cols = thresholds.
    """
    if df.empty:
        return pd.DataFrame()
    
    results = []
    for period in sorted(df["period"].unique()):
        period_df = df[df["period"] == period]
        row = {"period": f"T+{period}d", "n": len(period_df)}
        
        for thresh in thresholds:
            col = f"hit_{int(thresh)}pct"
            if col in period_df.columns:
                hit_rate = period_df[col].mean() * 100
                row[f"+{int(thresh)}%"] = f"{hit_rate:.1f}%"
        
        # Also add average return
        row["Avg Return"] = f"{period_df['return_pct'].mean():.1f}%"
        row["Avg Max Return"] = f"{period_df['max_return_pct'].mean():.1f}%"
        
        results.append(row)
    
    return pd.DataFrame(results)


def compute_strategy_comparison(df: pd.DataFrame, period: int = PRIMARY_PERIOD) -> pd.DataFrame:
    """
    Compare strategies (weekly, pro30, movers) for a specific holding period.
    """
    if df.empty:
        return pd.DataFrame()
    
    period_df = df[df["period"] == period].copy()
    
    strategies = [
        ("All Picks", period_df),
        ("Weekly Top 5", period_df[period_df["in_weekly_top5"]]),
        ("Pro30", period_df[period_df["in_pro30"]]),
        ("Movers", period_df[period_df["in_movers"]]),
    ]
    
    results = []
    for name, sub_df in strategies:
        if sub_df.empty:
            continue
        
        results.append({
            "Strategy": name,
            "N": len(sub_df),
            "Hit +5%": f"{sub_df['hit_5pct'].mean() * 100:.1f}%" if "hit_5pct" in sub_df.columns else "—",
            "Hit +10%": f"{sub_df['hit_10pct'].mean() * 100:.1f}%" if "hit_10pct" in sub_df.columns else "—",
            "Hit +15%": f"{sub_df['hit_15pct'].mean() * 100:.1f}%" if "hit_15pct" in sub_df.columns else "—",
            "Avg Return": f"{sub_df['return_pct'].mean():.1f}%",
            "Avg Max": f"{sub_df['max_return_pct'].mean():.1f}%",
            "Avg DD": f"{sub_df['max_drawdown_pct'].mean():.1f}%",
            "Win Rate": f"{(sub_df['return_pct'] > 0).mean() * 100:.1f}%",
        })
    
    return pd.DataFrame(results)


def compute_daily_trend(df: pd.DataFrame, period: int = PRIMARY_PERIOD) -> pd.DataFrame:
    """
    Compute hit rate trend by scan date.
    """
    if df.empty:
        return pd.DataFrame()
    
    period_df = df[df["period"] == period].copy()
    
    results = []
    for date, g in period_df.groupby("scan_date"):
        results.append({
            "Date": date,
            "N": len(g),
            "Hit Rate +10%": g["hit_10pct"].mean() * 100 if "hit_10pct" in g.columns else None,
            "Avg Return": g["return_pct"].mean(),
            "Avg Max Return": g["max_return_pct"].mean(),
            "Win Rate": (g["return_pct"] > 0).mean() * 100,
        })
    
    return pd.DataFrame(results)


def compute_factor_attribution(df: pd.DataFrame, period: int = PRIMARY_PERIOD) -> dict:
    """
    Analyze which factors predict success.
    """
    if df.empty:
        return {}
    
    period_df = df[df["period"] == period].copy()
    
    results = {}
    
    # Weekly rank analysis
    weekly_df = period_df[period_df["in_weekly_top5"] & period_df["weekly_rank"].notna()].copy()
    if not weekly_df.empty:
        rank_perf = weekly_df.groupby("weekly_rank").agg({
            "hit_10pct": "mean",
            "return_pct": "mean",
            "ticker": "count",
        }).rename(columns={"ticker": "n", "hit_10pct": "hit_rate", "return_pct": "avg_return"})
        results["by_weekly_rank"] = rank_perf
    
    # Composite score buckets
    score_df = period_df[period_df["composite_score"].notna()].copy()
    if not score_df.empty:
        score_df["score_bucket"] = pd.cut(
            score_df["composite_score"],
            bins=[0, 4.5, 5.0, 5.5, 6.0, 10],
            labels=["<4.5", "4.5-5.0", "5.0-5.5", "5.5-6.0", "≥6.0"],
        )
        score_perf = score_df.groupby("score_bucket", observed=True).agg({
            "hit_10pct": "mean",
            "return_pct": "mean",
            "ticker": "count",
        }).rename(columns={"ticker": "n", "hit_10pct": "hit_rate", "return_pct": "avg_return"})
        results["by_composite_score"] = score_perf
    
    # Source analysis
    source_perf = period_df.groupby("source").agg({
        "hit_10pct": "mean",
        "return_pct": "mean",
        "ticker": "count",
    }).rename(columns={"ticker": "n", "hit_10pct": "hit_rate", "return_pct": "avg_return"})
    results["by_source"] = source_perf
    
    return results


print(f"✅ Analytics functions ready")

✅ Analytics functions ready


In [6]:
# ═══════════════════════════════════════════════════════════════════════════════
# VISUALIZATION
# ═══════════════════════════════════════════════════════════════════════════════

def plot_hit_rate_heatmap(df: pd.DataFrame):
    """
    Heatmap: Holding Period x Threshold hit rates.
    """
    if df.empty:
        print("No data to plot")
        return
    
    # Build matrix
    periods = sorted(df["period"].unique())
    thresholds = [5, 10, 15]
    
    matrix = []
    for period in periods:
        period_df = df[df["period"] == period]
        row = []
        for thresh in thresholds:
            col = f"hit_{thresh}pct"
            if col in period_df.columns:
                row.append(period_df[col].mean() * 100)
            else:
                row.append(0)
        matrix.append(row)
    
    fig = go.Figure(data=go.Heatmap(
        z=matrix,
        x=[f"+{t}%" for t in thresholds],
        y=[f"T+{p}d" for p in periods],
        colorscale="RdYlGn",
        text=[[f"{v:.1f}%" for v in row] for row in matrix],
        texttemplate="%{text}",
        textfont={"size": 14},
        hovertemplate="Period: %{y}<br>Threshold: %{x}<br>Hit Rate: %{z:.1f}%<extra></extra>",
    ))
    
    fig.update_layout(
        title="Hit Rate Matrix: Holding Period × Target Gain",
        xaxis_title="Target Gain Threshold",
        yaxis_title="Holding Period",
        template="plotly_white",
        height=400,
        width=600,
    )
    
    fig.show()


def plot_strategy_comparison(df: pd.DataFrame, period: int = PRIMARY_PERIOD):
    """
    Bar chart comparing strategies.
    """
    if df.empty:
        print("No data to plot")
        return
    
    period_df = df[df["period"] == period].copy()
    
    strategies = {
        "All": period_df,
        "Weekly Top 5": period_df[period_df["in_weekly_top5"]],
        "Pro30": period_df[period_df["in_pro30"]],
        "Movers": period_df[period_df["in_movers"]],
    }
    
    data = []
    for name, sub_df in strategies.items():
        if sub_df.empty:
            continue
        data.append({
            "Strategy": name,
            "Hit +10%": sub_df["hit_10pct"].mean() * 100 if "hit_10pct" in sub_df.columns else 0,
            "Avg Return": sub_df["return_pct"].mean(),
            "Win Rate": (sub_df["return_pct"] > 0).mean() * 100,
        })
    
    if not data:
        return
    
    chart_df = pd.DataFrame(data)
    
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=["Hit +10% Rate", "Avg Return (%)", "Win Rate (%)"]
    )
    
    colors = ["#636EFA", "#00CC96", "#EF553B", "#AB63FA"]
    
    for i, metric in enumerate(["Hit +10%", "Avg Return", "Win Rate"]):
        fig.add_trace(
            go.Bar(
                x=chart_df["Strategy"],
                y=chart_df[metric],
                marker_color=colors[:len(chart_df)],
                text=[f"{v:.1f}%" for v in chart_df[metric]],
                textposition="outside",
                showlegend=False,
            ),
            row=1, col=i+1
        )
    
    fig.update_layout(
        title=f"Strategy Comparison (T+{period} days)",
        template="plotly_white",
        height=400,
        width=1000,
    )
    
    fig.show()


def plot_trend_over_time(df: pd.DataFrame, period: int = PRIMARY_PERIOD):
    """
    Line chart showing model performance trend over time.
    """
    trend_df = compute_daily_trend(df, period)
    
    if trend_df.empty:
        print("No trend data to plot")
        return
    
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=["Hit Rate +10% Over Time", "Average Return Over Time"],
        shared_xaxes=True,
        vertical_spacing=0.12,
    )
    
    # Hit rate trend
    fig.add_trace(
        go.Scatter(
            x=trend_df["Date"],
            y=trend_df["Hit Rate +10%"],
            mode="lines+markers",
            name="Hit Rate",
            line=dict(color="#00CC96", width=2),
            marker=dict(size=8),
        ),
        row=1, col=1
    )
    
    # Add rolling average
    if len(trend_df) > 3:
        trend_df["Hit_Rate_MA"] = trend_df["Hit Rate +10%"].rolling(3, min_periods=1).mean()
        fig.add_trace(
            go.Scatter(
                x=trend_df["Date"],
                y=trend_df["Hit_Rate_MA"],
                mode="lines",
                name="3-day MA",
                line=dict(color="#636EFA", width=2, dash="dash"),
            ),
            row=1, col=1
        )
    
    # Return trend
    colors = ["#00CC96" if r > 0 else "#EF553B" for r in trend_df["Avg Return"]]
    fig.add_trace(
        go.Bar(
            x=trend_df["Date"],
            y=trend_df["Avg Return"],
            marker_color=colors,
            name="Avg Return",
        ),
        row=2, col=1
    )
    
    fig.update_layout(
        title=f"Model Performance Trend (T+{period} days)",
        template="plotly_white",
        height=600,
        width=1000,
        showlegend=True,
    )
    
    fig.update_yaxes(title_text="Hit Rate (%)", row=1, col=1)
    fig.update_yaxes(title_text="Avg Return (%)", row=2, col=1)
    fig.update_xaxes(title_text="Scan Date", row=2, col=1)
    
    fig.show()


def plot_factor_analysis(attribution: dict):
    """
    Visualize factor attribution analysis.
    """
    if not attribution:
        print("No attribution data")
        return
    
    n_plots = len(attribution)
    fig = make_subplots(
        rows=1, cols=n_plots,
        subplot_titles=list(attribution.keys()),
    )
    
    for i, (name, data) in enumerate(attribution.items()):
        if data.empty:
            continue
        
        data = data.reset_index()
        x_col = data.columns[0]
        
        fig.add_trace(
            go.Bar(
                x=data[x_col].astype(str),
                y=data["hit_rate"] * 100,
                text=[f"{v:.1f}%" for v in data["hit_rate"] * 100],
                textposition="outside",
                name=name,
                showlegend=False,
            ),
            row=1, col=i+1
        )
    
    fig.update_layout(
        title="Factor Attribution: Hit Rate by Factor",
        template="plotly_white",
        height=400,
        width=400 * n_plots,
    )
    
    fig.show()


print(f"✅ Visualization functions ready")

✅ Visualization functions ready


In [7]:
# ═══════════════════════════════════════════════════════════════════════════════
# MODEL QUALITY SCORECARD
# ═══════════════════════════════════════════════════════════════════════════════

def generate_model_scorecard(df: pd.DataFrame) -> dict:
    """
    Generate a comprehensive model quality scorecard.
    """
    if df.empty:
        return {"status": "No data"}
    
    primary_df = df[df["period"] == PRIMARY_PERIOD].copy()
    
    scorecard = {
        "data_summary": {
            "total_observations": len(primary_df),
            "scan_dates": primary_df["scan_date"].nunique(),
            "unique_tickers": primary_df["ticker"].nunique(),
            "date_range": f"{primary_df['scan_date'].min()} → {primary_df['scan_date'].max()}",
        },
        "primary_kpi": {
            "metric": f"Hit +{PRIMARY_THRESHOLD}% within T+{PRIMARY_PERIOD} days",
            "hit_rate": primary_df["hit_10pct"].mean() if "hit_10pct" in primary_df.columns else None,
            "avg_return": primary_df["return_pct"].mean(),
            "avg_max_return": primary_df["max_return_pct"].mean(),
            "win_rate": (primary_df["return_pct"] > 0).mean(),
            "avg_drawdown": primary_df["max_drawdown_pct"].mean(),
        },
        "strategy_ranking": [],
        "model_health": "Unknown",
        "recommendations": [],
    }
    
    # Strategy ranking
    for name, mask in [("weekly_top5", primary_df["in_weekly_top5"]), 
                       ("pro30", primary_df["in_pro30"]),
                       ("movers", primary_df["in_movers"])]:
        sub = primary_df[mask]
        if len(sub) > 0:
            scorecard["strategy_ranking"].append({
                "strategy": name,
                "n": len(sub),
                "hit_rate": sub["hit_10pct"].mean() if "hit_10pct" in sub.columns else None,
                "avg_return": sub["return_pct"].mean(),
            })
    
    # Sort by hit rate
    scorecard["strategy_ranking"] = sorted(
        scorecard["strategy_ranking"], 
        key=lambda x: x.get("hit_rate") or 0, 
        reverse=True
    )
    
    # Model health assessment
    hit_rate = scorecard["primary_kpi"]["hit_rate"] or 0
    win_rate = scorecard["primary_kpi"]["win_rate"] or 0
    
    if hit_rate >= 0.35 and win_rate >= 0.55:
        scorecard["model_health"] = "🟢 Excellent"
    elif hit_rate >= 0.25 and win_rate >= 0.45:
        scorecard["model_health"] = "🟡 Good"
    elif hit_rate >= 0.15:
        scorecard["model_health"] = "🟠 Needs Attention"
    else:
        scorecard["model_health"] = "🔴 Poor"
    
    # Recommendations
    if hit_rate < 0.25:
        scorecard["recommendations"].append("Consider tightening quality filters (min_technical_score, min_composite_score)")
    
    if scorecard["strategy_ranking"]:
        best = scorecard["strategy_ranking"][0]
        worst = scorecard["strategy_ranking"][-1]
        if best["hit_rate"] and worst["hit_rate"] and best["hit_rate"] > worst["hit_rate"] * 1.5:
            scorecard["recommendations"].append(f"Focus on {best['strategy']} strategy (significantly outperforms others)")
    
    recent_df = primary_df[primary_df["scan_date"] >= (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")]
    if len(recent_df) > 0:
        recent_hit = recent_df["hit_10pct"].mean() if "hit_10pct" in recent_df.columns else 0
        if recent_hit < hit_rate * 0.7:
            scorecard["recommendations"].append("⚠️ Recent performance declining - review market regime")
    
    return scorecard


def display_scorecard(scorecard: dict):
    """Pretty print the scorecard."""
    print("\n" + "═" * 70)
    print("📊 MODEL QUALITY SCORECARD")
    print("═" * 70)
    
    summary = scorecard.get("data_summary", {})
    print(f"\n📈 Data Summary:")
    print(f"   • Observations: {summary.get('total_observations', 0):,}")
    print(f"   • Scan Dates: {summary.get('scan_dates', 0)}")
    print(f"   • Unique Tickers: {summary.get('unique_tickers', 0)}")
    print(f"   • Date Range: {summary.get('date_range', 'N/A')}")
    
    kpi = scorecard.get("primary_kpi", {})
    print(f"\n🎯 Primary KPI ({kpi.get('metric', 'N/A')}):")
    print(f"   • Hit Rate: {kpi.get('hit_rate', 0) * 100:.1f}%")
    print(f"   • Win Rate: {kpi.get('win_rate', 0) * 100:.1f}%")
    print(f"   • Avg Return: {kpi.get('avg_return', 0):.1f}%")
    print(f"   • Avg Max Return: {kpi.get('avg_max_return', 0):.1f}%")
    print(f"   • Avg Drawdown: {kpi.get('avg_drawdown', 0):.1f}%")
    
    print(f"\n🏆 Strategy Ranking (by Hit Rate):")
    for i, s in enumerate(scorecard.get("strategy_ranking", []), 1):
        hr = s.get('hit_rate', 0) or 0
        ar = s.get('avg_return', 0) or 0
        print(f"   {i}. {s['strategy']}: Hit={hr*100:.1f}% | Return={ar:.1f}% | N={s['n']}")
    
    print(f"\n💊 Model Health: {scorecard.get('model_health', 'Unknown')}")
    
    recs = scorecard.get("recommendations", [])
    if recs:
        print(f"\n💡 Recommendations:")
        for r in recs:
            print(f"   • {r}")
    
    print("\n" + "═" * 70)


print(f"✅ Model scorecard functions ready")

✅ Model scorecard functions ready


In [8]:
# ═══════════════════════════════════════════════════════════════════════════════
# EXPORT & PERSISTENCE
# ═══════════════════════════════════════════════════════════════════════════════

def save_backtest_results(
    df: pd.DataFrame,
    scorecard: dict,
    output_dir: Path = OUTPUTS_ROOT / "performance",
) -> dict[str, str]:
    """
    Save backtest results and scorecard to disk.
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    timestamp = datetime.now().strftime("%Y-%m-%d")
    paths = {}
    
    # Full detail
    detail_path = output_dir / f"backtest_detail_{timestamp}.csv"
    df.to_csv(detail_path, index=False)
    paths["detail"] = str(detail_path)
    
    # Hit rate matrix
    matrix = compute_hit_rate_matrix(df)
    matrix_path = output_dir / f"hit_rate_matrix_{timestamp}.csv"
    matrix.to_csv(matrix_path, index=False)
    paths["matrix"] = str(matrix_path)
    
    # Strategy comparison
    strategy = compute_strategy_comparison(df)
    strategy_path = output_dir / f"strategy_comparison_{timestamp}.csv"
    strategy.to_csv(strategy_path, index=False)
    paths["strategy"] = str(strategy_path)
    
    # Daily trend
    trend = compute_daily_trend(df)
    trend_path = output_dir / f"daily_trend_{timestamp}.csv"
    trend.to_csv(trend_path, index=False)
    paths["trend"] = str(trend_path)
    
    # Scorecard
    scorecard_path = output_dir / f"scorecard_{timestamp}.json"
    with open(scorecard_path, "w") as f:
        json.dump(scorecard, f, indent=2, default=str)
    paths["scorecard"] = str(scorecard_path)
    
    logger.info(f"Saved backtest results to {output_dir}")
    return paths


print(f"✅ Export functions ready")

✅ Export functions ready


---

# 🚀 RUN FULL ANALYSIS

Execute the cells below to run the complete model validation.

In [9]:
# ═══════════════════════════════════════════════════════════════════════════════
# 1. RUN BACKTEST
# ═══════════════════════════════════════════════════════════════════════════════

# Optional: Set date range (leave as None for all available data)
START_DATE = None  # e.g., "2025-12-01"
END_DATE = None    # e.g., "2026-01-10"

print("🔄 Running multi-period backtest...")
backtest_df = run_full_backtest(
    start_date=START_DATE,
    end_date=END_DATE,
    holding_periods=HOLDING_PERIODS,
    hit_thresholds=HIT_THRESHOLDS,
)

if not backtest_df.empty:
    print(f"\n✅ Backtest complete!")
    print(f"   • Total observations: {len(backtest_df):,}")
    print(f"   • Holding periods tested: {sorted(backtest_df['period'].unique())}")
else:
    print("❌ No backtest data generated")

2026-01-15 00:09:20,195 | INFO | Found 33 scan dates: 2025-11-17 → 2026-01-13
2026-01-15 00:09:20,252 | INFO | Total unique tickers: 106
2026-01-15 00:09:20,253 | INFO | Downloading prices for 106 tickers: 2025-11-17 → 2026-01-15
2026-01-15 00:09:20,253 | INFO | Using Polygon.io as primary data source...


🔄 Running multi-period backtest...


2026-01-15 00:09:52,145 | INFO | Polygon: 106/106 tickers succeeded
2026-01-15 00:09:52,255 | INFO | Backtest complete: 620 observations



✅ Backtest complete!
   • Total observations: 620
   • Holding periods tested: [5, 7, 10, 14]


In [10]:
# ═══════════════════════════════════════════════════════════════════════════════
# 2. HIT RATE MATRIX
# ═══════════════════════════════════════════════════════════════════════════════

print("📊 Hit Rate Matrix (Period × Threshold):")
print("="*60)
matrix = compute_hit_rate_matrix(backtest_df)
display(matrix)

plot_hit_rate_heatmap(backtest_df)

📊 Hit Rate Matrix (Period × Threshold):


,period,n,+5%,+10%,+15%,Avg Return,Avg Max Return
0,T+5d,155,54.2%,27.7%,12.3%,3.1%,7.5%
1,T+7d,155,59.4%,34.8%,16.8%,3.2%,8.8%
2,T+10d,155,61.9%,39.4%,19.4%,2.9%,9.6%
3,T+14d,155,63.9%,42.6%,20.6%,2.6%,10.1%


In [11]:
# ═══════════════════════════════════════════════════════════════════════════════
# 3. STRATEGY COMPARISON
# ═══════════════════════════════════════════════════════════════════════════════

print(f"📊 Strategy Comparison (T+{PRIMARY_PERIOD} days):")
print("="*60)
strategy_df = compute_strategy_comparison(backtest_df, period=PRIMARY_PERIOD)
display(strategy_df)

plot_strategy_comparison(backtest_df, period=PRIMARY_PERIOD)

📊 Strategy Comparison (T+7 days):


,Strategy,N,Hit +5%,Hit +10%,Hit +15%,Avg Return,Avg Max,Avg DD,Win Rate
0,All Picks,155,59.4%,34.8%,16.8%,3.2%,8.8%,-4.5%,65.2%
1,Weekly Top 5,105,55.2%,28.6%,13.3%,3.3%,7.9%,-3.7%,66.7%
2,Pro30,48,68.8%,50.0%,25.0%,2.9%,10.9%,-6.3%,60.4%
3,Movers,2,50.0%,0.0%,0.0%,3.3%,4.4%,-3.0%,100.0%


In [12]:
# ═══════════════════════════════════════════════════════════════════════════════
# 4. PERFORMANCE TREND OVER TIME
# ═══════════════════════════════════════════════════════════════════════════════

print("📈 Model Performance Trend:")
print("="*60)
trend_df = compute_daily_trend(backtest_df, period=PRIMARY_PERIOD)
display(trend_df)

plot_trend_over_time(backtest_df, period=PRIMARY_PERIOD)

📈 Model Performance Trend:


,Date,N,Hit Rate +10%,Avg Return,Avg Max Return,Win Rate
0,2025-11-17,3,33.333333,6.703333,8.526667,66.666667
1,2025-11-19,2,50.000000,1.060000,9.505000,50.000000
2,2025-11-20,1,100.000000,9.810000,19.900000,100.000000
3,2025-11-21,6,16.666667,8.053333,10.746667,100.000000
4,2025-11-24,8,37.500000,6.295000,14.057500,75.000000
5,2025-11-25,1,100.000000,2.810000,12.490000,100.000000
6,2025-11-27,7,28.571429,5.785714,8.937143,71.428571
7,2025-11-28,5,20.000000,5.340000,7.108000,80.000000
8,2025-12-01,6,33.333333,1.831667,5.905000,50.000000
9,2025-12-02,6,33.333333,8.538333,11.205000,83.333333


In [13]:
# ═══════════════════════════════════════════════════════════════════════════════
# 5. FACTOR ATTRIBUTION ANALYSIS
# ═══════════════════════════════════════════════════════════════════════════════

print("🔍 Factor Attribution Analysis:")
print("="*60)

attribution = compute_factor_attribution(backtest_df, period=PRIMARY_PERIOD)

for name, data in attribution.items():
    if not data.empty:
        print(f"\n{name}:")
        display(data.round(3))

plot_factor_analysis(attribution)

🔍 Factor Attribution Analysis:

by_weekly_rank:


,hit_rate,avg_return,n
weekly_rank,,,
1.0,0.381,6.083,21
2.0,0.190,2.019,21
3.0,0.286,3.756,21
4.0,0.286,1.548,21
5.0,0.286,2.976,21



by_composite_score:


,hit_rate,avg_return,n
score_bucket,,,
<4.5,0.400,3.250,5
4.5-5.0,0.185,-0.335,27
5.0-5.5,0.308,4.008,26
5.5-6.0,0.500,27.360,2



by_source:


,hit_rate,avg_return,n
source,,,
30d_momentum,0.500,2.893,48
movers,0.000,3.305,2
weekly_top5,0.286,3.276,105


In [14]:
# ═══════════════════════════════════════════════════════════════════════════════
# 6. MODEL QUALITY SCORECARD
# ═══════════════════════════════════════════════════════════════════════════════

scorecard = generate_model_scorecard(backtest_df)
display_scorecard(scorecard)


══════════════════════════════════════════════════════════════════════
📊 MODEL QUALITY SCORECARD
══════════════════════════════════════════════════════════════════════

📈 Data Summary:
   • Observations: 155
   • Scan Dates: 28
   • Unique Tickers: 105
   • Date Range: 2025-11-17 → 2026-01-13

🎯 Primary KPI (Hit +10.0% within T+7 days):
   • Hit Rate: 34.8%
   • Win Rate: 65.2%
   • Avg Return: 3.2%
   • Avg Max Return: 8.8%
   • Avg Drawdown: -4.5%

🏆 Strategy Ranking (by Hit Rate):
   1. pro30: Hit=50.0% | Return=2.9% | N=48
   2. weekly_top5: Hit=28.6% | Return=3.3% | N=105
   3. movers: Hit=0.0% | Return=3.3% | N=2

💊 Model Health: 🟡 Good

💡 Recommendations:
   • ⚠️ Recent performance declining - review market regime

══════════════════════════════════════════════════════════════════════


In [15]:
# ═══════════════════════════════════════════════════════════════════════════════
# 7. SAVE RESULTS
# ═══════════════════════════════════════════════════════════════════════════════

if not backtest_df.empty:
    saved_paths = save_backtest_results(backtest_df, scorecard)
    print("\n💾 Saved artifacts:")
    for name, path in saved_paths.items():
        print(f"   • {name}: {path}")
else:
    print("No data to save")

2026-01-15 00:09:52,831 | INFO | Saved backtest results to outputs/performance



💾 Saved artifacts:
   • detail: outputs/performance/backtest_detail_2026-01-15.csv
   • matrix: outputs/performance/hit_rate_matrix_2026-01-15.csv
   • strategy: outputs/performance/strategy_comparison_2026-01-15.csv
   • trend: outputs/performance/daily_trend_2026-01-15.csv
   • scorecard: outputs/performance/scorecard_2026-01-15.json


---

# 🔬 DEEP DIVE: Compare Holding Periods

Which holding period works best for your picks?

In [16]:
# ═══════════════════════════════════════════════════════════════════════════════
# OPTIMAL HOLDING PERIOD ANALYSIS
# ═══════════════════════════════════════════════════════════════════════════════

def find_optimal_holding_period(df: pd.DataFrame) -> dict:
    """
    Find the optimal holding period based on risk-adjusted returns.
    """
    results = []
    
    for period in sorted(df["period"].unique()):
        period_df = df[df["period"] == period]
        
        avg_return = period_df["return_pct"].mean()
        std_return = period_df["return_pct"].std()
        hit_rate = period_df["hit_10pct"].mean() if "hit_10pct" in period_df.columns else 0
        win_rate = (period_df["return_pct"] > 0).mean()
        avg_drawdown = abs(period_df["max_drawdown_pct"].mean())
        
        # Sharpe-like ratio (simplified)
        sharpe = avg_return / std_return if std_return > 0 else 0
        
        # Return/Drawdown ratio
        return_dd_ratio = avg_return / avg_drawdown if avg_drawdown > 0 else 0
        
        results.append({
            "Period": f"T+{period}d",
            "N": len(period_df),
            "Avg Return": round(avg_return, 2),
            "Std Dev": round(std_return, 2),
            "Sharpe": round(sharpe, 3),
            "Hit +10%": f"{hit_rate*100:.1f}%",
            "Win Rate": f"{win_rate*100:.1f}%",
            "Avg DD": round(period_df["max_drawdown_pct"].mean(), 2),
            "Ret/DD": round(return_dd_ratio, 3),
        })
    
    return pd.DataFrame(results)


optimal_df = find_optimal_holding_period(backtest_df)
print("📊 Holding Period Comparison:")
print("="*80)
display(optimal_df)

# Find recommended period
if not optimal_df.empty:
    best_sharpe_idx = optimal_df["Sharpe"].idxmax()
    best_period = optimal_df.loc[best_sharpe_idx, "Period"]
    print(f"\n💡 Recommended holding period (best Sharpe): {best_period}")

📊 Holding Period Comparison:


,Period,N,Avg Return,Std Dev,Sharpe,Hit +10%,Win Rate,Avg DD,Ret/DD
0,T+5d,155,3.06,8.13,0.376,27.7%,67.7%,-3.84,0.796
1,T+7d,155,3.16,9.67,0.327,34.8%,65.2%,-4.48,0.705
2,T+10d,155,2.87,10.19,0.282,39.4%,60.6%,-5.23,0.549
3,T+14d,155,2.62,10.38,0.252,42.6%,58.1%,-5.59,0.469



💡 Recommended holding period (best Sharpe): T+5d


In [17]:
# ═══════════════════════════════════════════════════════════════════════════════
# VISUALIZE ALL PERIODS
# ═══════════════════════════════════════════════════════════════════════════════

def plot_all_periods(df: pd.DataFrame):
    """Compare metrics across all holding periods."""
    if df.empty:
        return
    
    periods = sorted(df["period"].unique())
    
    data = []
    for period in periods:
        period_df = df[df["period"] == period]
        data.append({
            "Period": f"T+{period}d",
            "Hit +10%": period_df["hit_10pct"].mean() * 100 if "hit_10pct" in period_df.columns else 0,
            "Avg Return": period_df["return_pct"].mean(),
            "Win Rate": (period_df["return_pct"] > 0).mean() * 100,
        })
    
    chart_df = pd.DataFrame(data)
    
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=chart_df["Period"],
        y=chart_df["Hit +10%"],
        name="Hit +10%",
        marker_color="#00CC96",
    ))
    
    fig.add_trace(go.Bar(
        x=chart_df["Period"],
        y=chart_df["Win Rate"],
        name="Win Rate",
        marker_color="#636EFA",
    ))
    
    fig.add_trace(go.Scatter(
        x=chart_df["Period"],
        y=chart_df["Avg Return"],
        name="Avg Return",
        mode="lines+markers",
        yaxis="y2",
        line=dict(color="#EF553B", width=3),
        marker=dict(size=10),
    ))
    
    fig.update_layout(
        title="Performance by Holding Period",
        xaxis_title="Holding Period",
        yaxis=dict(title="Rate (%)", side="left"),
        yaxis2=dict(title="Avg Return (%)", side="right", overlaying="y"),
        template="plotly_white",
        height=500,
        width=800,
        barmode="group",
        legend=dict(orientation="h", y=1.1),
    )
    
    fig.show()


plot_all_periods(backtest_df)

---

# 📋 QUICK DAILY CHECK

Run this after each daily scan to track recent performance.

In [18]:
# ═══════════════════════════════════════════════════════════════════════════════
# QUICK DAILY PERFORMANCE CHECK
# ═══════════════════════════════════════════════════════════════════════════════

def quick_daily_check(lookback_days: int = 14):
    """
    Quick check of recent model performance.
    Run this after each daily scan.
    """
    cutoff = (datetime.now() - timedelta(days=lookback_days)).strftime("%Y-%m-%d")
    
    print(f"\n🔄 Running quick check (last {lookback_days} days)...")
    
    recent_df = run_full_backtest(
        start_date=cutoff,
        holding_periods=[5, 7],  # Quick check uses shorter periods
        hit_thresholds=[10.0],
    )
    
    if recent_df.empty:
        print("No recent data available")
        return None
    
    print("\n" + "═" * 60)
    print(f"📊 QUICK DAILY CHECK ({lookback_days}-Day Window)")
    print("═" * 60)
    
    for period in sorted(recent_df["period"].unique()):
        period_df = recent_df[recent_df["period"] == period]
        hit_rate = period_df["hit_10pct"].mean() * 100 if "hit_10pct" in period_df.columns else 0
        avg_return = period_df["return_pct"].mean()
        win_rate = (period_df["return_pct"] > 0).mean() * 100
        
        status = "🟢" if hit_rate >= 25 else "🟡" if hit_rate >= 15 else "🔴"
        
        print(f"\n{status} T+{period} days:")
        print(f"   Hit +10%: {hit_rate:.1f}% | Win Rate: {win_rate:.1f}% | Avg Return: {avg_return:.1f}%")
    
    # Show recent picks performance
    print("\n📋 Recent Picks Performance:")
    recent_picks = recent_df[recent_df["period"] == 7].sort_values("scan_date", ascending=False)
    
    for date in recent_picks["scan_date"].unique()[:5]:
        day_df = recent_picks[recent_picks["scan_date"] == date]
        hits = day_df["hit_10pct"].sum() if "hit_10pct" in day_df.columns else 0
        total = len(day_df)
        avg = day_df["return_pct"].mean()
        print(f"   {date}: {int(hits)}/{total} hits | Avg: {avg:+.1f}%")
    
    print("\n" + "═" * 60)
    
    return recent_df


# Run quick check
recent_results = quick_daily_check(lookback_days=14)

2026-01-15 00:09:52,876 | INFO | Found 10 scan dates: 2026-01-01 → 2026-01-13
2026-01-15 00:09:52,898 | INFO | Total unique tickers: 51
2026-01-15 00:09:52,899 | INFO | Downloading prices for 51 tickers: 2026-01-01 → 2026-01-15
2026-01-15 00:09:52,899 | INFO | Using Polygon.io as primary data source...



🔄 Running quick check (last 14 days)...


2026-01-15 00:10:08,845 | INFO | Polygon: 51/51 tickers succeeded
2026-01-15 00:10:08,879 | INFO | Backtest complete: 120 observations



════════════════════════════════════════════════════════════
📊 QUICK DAILY CHECK (14-Day Window)
════════════════════════════════════════════════════════════

🟡 T+5 days:
   Hit +10%: 20.0% | Win Rate: 65.0% | Avg Return: 2.1%

🟢 T+7 days:
   Hit +10%: 26.7% | Win Rate: 66.7% | Avg Return: 2.1%

📋 Recent Picks Performance:
   2026-01-13: 0/2 hits | Avg: +0.7%
   2026-01-12: 1/7 hits | Avg: +0.6%
   2026-01-09: 1/12 hits | Avg: +1.3%
   2026-01-08: 0/5 hits | Avg: -3.0%
   2026-01-07: 1/7 hits | Avg: +3.1%

════════════════════════════════════════════════════════════


---

# 🎯 Model Improvement Suggestions

Based on the backtest results, here are automatic suggestions for improving the model.

In [19]:
# ═══════════════════════════════════════════════════════════════════════════════
# AUTOMATED MODEL IMPROVEMENT SUGGESTIONS
# ═══════════════════════════════════════════════════════════════════════════════

def generate_improvement_suggestions(df: pd.DataFrame, period: int = PRIMARY_PERIOD) -> list[dict]:
    """
    Analyze backtest results and generate specific improvement suggestions.
    """
    suggestions = []
    period_df = df[df["period"] == period].copy()
    
    if period_df.empty:
        return suggestions
    
    # 1. Check if certain strategies consistently outperform
    weekly_hit = period_df[period_df["in_weekly_top5"]]["hit_10pct"].mean() if "hit_10pct" in period_df.columns else 0
    pro30_hit = period_df[period_df["in_pro30"]]["hit_10pct"].mean() if "hit_10pct" in period_df.columns else 0
    movers_hit = period_df[period_df["in_movers"]]["hit_10pct"].mean() if "hit_10pct" in period_df.columns else 0
    
    if pro30_hit > weekly_hit * 1.3 and pro30_hit > 0.25:
        suggestions.append({
            "category": "Strategy",
            "priority": "High",
            "suggestion": f"Pro30 significantly outperforms Weekly ({pro30_hit*100:.1f}% vs {weekly_hit*100:.1f}%). Consider increasing Pro30 allocation.",
            "config_change": "Increase pro30 weight in hybrid analysis",
        })
    
    # 2. Check weekly rank performance
    weekly_df = period_df[period_df["in_weekly_top5"] & period_df["weekly_rank"].notna()]
    if not weekly_df.empty:
        rank_perf = weekly_df.groupby("weekly_rank")["hit_10pct"].mean()
        if len(rank_perf) >= 3:
            top_3_hit = rank_perf.iloc[:3].mean() if len(rank_perf) >= 3 else rank_perf.mean()
            bottom_hit = rank_perf.iloc[3:].mean() if len(rank_perf) > 3 else 0
            
            if top_3_hit > bottom_hit * 1.5 and top_3_hit > 0.25:
                suggestions.append({
                    "category": "Weekly Scanner",
                    "priority": "Medium",
                    "suggestion": f"Top 3 ranked picks significantly outperform ({top_3_hit*100:.1f}% vs {bottom_hit*100:.1f}%). Consider focusing on Top 3 only.",
                    "config_change": "Set top_n: 3 in weekly scanner config",
                })
    
    # 3. Check composite score threshold
    score_df = period_df[period_df["composite_score"].notna()]
    if not score_df.empty:
        high_score = score_df[score_df["composite_score"] >= 5.5]["hit_10pct"].mean() if "hit_10pct" in score_df.columns else 0
        low_score = score_df[score_df["composite_score"] < 5.0]["hit_10pct"].mean() if "hit_10pct" in score_df.columns else 0
        
        if high_score > low_score * 1.5 and high_score > 0.25:
            suggestions.append({
                "category": "Quality Filters",
                "priority": "High",
                "suggestion": f"High composite scores (≥5.5) significantly outperform ({high_score*100:.1f}% vs {low_score*100:.1f}%). Raise minimum threshold.",
                "config_change": "quality_filters_weekly.min_composite_score: 5.5",
            })
    
    # 4. Check for declining performance trend
    trend_df = compute_daily_trend(df, period)
    if len(trend_df) >= 5:
        recent_hit = trend_df.tail(3)["Hit Rate +10%"].mean()
        earlier_hit = trend_df.head(len(trend_df) - 3)["Hit Rate +10%"].mean()
        
        if recent_hit < earlier_hit * 0.7 and earlier_hit > 15:
            suggestions.append({
                "category": "Market Regime",
                "priority": "High",
                "suggestion": f"Recent performance declining ({recent_hit:.1f}% vs {earlier_hit:.1f}% earlier). Market regime may have changed.",
                "config_change": "Review regime_gate settings; consider enabling ATR filter",
            })
    
    # 5. Check holding period optimization
    period_hits = {}
    for p in df["period"].unique():
        p_df = df[df["period"] == p]
        if "hit_10pct" in p_df.columns:
            period_hits[p] = p_df["hit_10pct"].mean()
    
    if period_hits:
        best_period = max(period_hits, key=period_hits.get)
        if best_period != PRIMARY_PERIOD and period_hits[best_period] > period_hits.get(PRIMARY_PERIOD, 0) * 1.2:
            suggestions.append({
                "category": "Holding Period",
                "priority": "Medium",
                "suggestion": f"T+{best_period}d has higher hit rate ({period_hits[best_period]*100:.1f}% vs {period_hits.get(PRIMARY_PERIOD, 0)*100:.1f}% at T+{PRIMARY_PERIOD}d). Consider adjusting target holding period.",
                "config_change": f"Set forward_trading_days: {best_period} in backtest config",
            })
    
    return suggestions


# Generate and display suggestions
if not backtest_df.empty:
    suggestions = generate_improvement_suggestions(backtest_df)
    
    print("\n" + "═" * 70)
    print("💡 MODEL IMPROVEMENT SUGGESTIONS")
    print("═" * 70)
    
    if suggestions:
        for i, s in enumerate(suggestions, 1):
            priority_icon = "🔴" if s["priority"] == "High" else "🟡" if s["priority"] == "Medium" else "🟢"
            print(f"\n{i}. [{priority_icon} {s['priority']}] {s['category']}")
            print(f"   {s['suggestion']}")
            print(f"   → Config: {s['config_change']}")
    else:
        print("\n✅ No immediate improvements identified. Model performing as expected.")
    
    print("\n" + "═" * 70)
else:
    print("No backtest data available for analysis")


══════════════════════════════════════════════════════════════════════
💡 MODEL IMPROVEMENT SUGGESTIONS
══════════════════════════════════════════════════════════════════════

1. [🔴 High] Strategy
   Pro30 significantly outperforms Weekly (50.0% vs 28.6%). Consider increasing Pro30 allocation.
   → Config: Increase pro30 weight in hybrid analysis

2. [🔴 High] Quality Filters
   High composite scores (≥5.5) significantly outperform (42.9% vs 20.0%). Raise minimum threshold.
   → Config: quality_filters_weekly.min_composite_score: 5.5

3. [🔴 High] Market Regime
   Recent performance declining (7.5% vs 45.4% earlier). Market regime may have changed.
   → Config: Review regime_gate settings; consider enabling ATR filter

4. [🟡 Medium] Holding Period
   T+14d has higher hit rate (42.6% vs 34.8% at T+7d). Consider adjusting target holding period.
   → Config: Set forward_trading_days: 14 in backtest config

══════════════════════════════════════════════════════════════════════
